In [1]:

#import libraries
import pandas as pd
import numpy as np

In [2]:
#load dataset
df = pd.read_excel("C:/Users/WNRC/Documents/Data  Analytics/Python/datasets/data_cleaning_in_python.xlsx")
df.head(5)

,customer_id,Product Name,Price,Quantity,Customer_Type,Order_Date,customer_email
0,1027,laptop,169.12,3,PREMIUM,NaT,bob@email.com
1,1046,Smartphone,1189.32,4,PREMIUM,NaT,charlie@email.com
2,1097,MOUSE,MISSING,2,Regular,2024-01-06,eve@email.com
3,1042,laptop,MISSING,5,Premium,2024-01-21,alice@email.com
4,1045,headphones,MISSING,3,regular,2024-01-04,charlie@email.com


In [3]:
# Let's also check for 'MISSING' and empty strings
print("Other forms of missing data:")
# missing valuels in Prices column
missing_Price= (df['Price']=="MISSING").sum()
print(f"missing_Price: {missing_Price}")


Other forms of missing data:
missing_Price: 118


## Step 2: Cleaning Missing Data

In [4]:
# First, let's replace 'MISSING' and empty strings with actual NaN values
df["Price"]= df["Price"].replace("MISSING",np.nan)
df["Price"].head(5)

C:\Users\WNRC\AppData\Local\Temp\ipykernel_20296\2278302494.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Price"]= df["Price"].replace("MISSING",np.nan)


0     169.12
1    1189.32
2        NaN
3        NaN
4        NaN
Name: Price, dtype: float64

In [5]:
df.isnull().sum()

customer_id         0
Product Name        0
Price             118
Quantity            0
Customer_Type       0
Order_Date        131
customer_email      0
dtype: int64

In [6]:
# Strategy 1: Drop rows with missing values (use carefully!)
df_drop_missing = df.dropna()
print(f"\nAfter dropping all missing values: {df_drop_missing.shape[0]} rows remain")


After dropping all missing values: 49 rows remain


In [7]:
# Strategy 2: Fill missing values with appropriate replacements
df_filled = df.copy()
# Fill missing prices with the median price (after converting to numeric)
# convert  to numeric
df_filled["Price"] = pd.to_numeric(df_filled["Price"], errors='coerce')
# find median price
median_price = df_filled["Price"].median()
# fill empty values with median price
df_filled["Price"]= df_filled["Price"].fillna(median_price)

In [8]:
df_filled.isnull().sum()

customer_id         0
Product Name        0
Price               0
Quantity            0
Customer_Type       0
Order_Date        131
customer_email      0
dtype: int64

In [9]:
df=df_filled.copy()

In [10]:
df["Order_Date"].head(5)

0          NaT
1          NaT
2   2024-01-06
3   2024-01-21
4   2024-01-04
Name: Order_Date, dtype: datetime64[ns]

In [11]:
# fill missing dates with a placeholder or forward fill
df_filled["Order_Date"] = df_filled["Order_Date"].fillna("2024-01-22")

In [12]:
df_filled.isnull().sum()

customer_id       0
Product Name      0
Price             0
Quantity          0
Customer_Type     0
Order_Date        0
customer_email    0
dtype: int64

️ Cleaning Column Names and Data Types

In [13]:
df_cleaned = df_filled.copy()

In [14]:
print(" Choosing the Right Strategy:") 
# For our retail data, let's make smart decisions: 
df_clean = df.copy() 
# Convert Price to numeric first 
df_clean['Price'] = pd.to_numeric(df_clean['Price'].replace('MISSING', np.nan), errors='coerce') 
# Fill missing price with median (makes business sense) 
df_clean['Price'] = df_clean['Price'].fillna(df_clean['Price'].median()) 
# Fill missing dates with the most recent date + 1 day 
df_clean['Order_Date'] = df_clean['Order_Date'].replace('', np.nan) 
df_clean['Order_Date'] = df_clean['Order_Date'].fillna('2024-01-22') 
print("✅ Strategic missing data handling complete!") 
print(df_clean.isnull().sum())


 Choosing the Right Strategy:
✅ Strategic missing data handling complete!
customer_id       0
Product Name      0
Price             0
Quantity          0
Customer_Type     0
Order_Date        0
customer_email    0
dtype: int64


In [15]:
df_clean.head(5)

,customer_id,Product Name,Price,Quantity,Customer_Type,Order_Date,customer_email
0,1027,laptop,169.120,3,PREMIUM,2024-01-22,bob@email.com
1,1046,Smartphone,1189.320,4,PREMIUM,2024-01-22,charlie@email.com
2,1097,MOUSE,527.895,2,Regular,2024-01-06,eve@email.com
3,1042,laptop,527.895,5,Premium,2024-01-21,alice@email.com
4,1045,headphones,527.895,3,regular,2024-01-04,charlie@email.com


Fixing Column Names

In [16]:
# Check current column names
print("Original columns:", df_clean.columns.tolist())

Original columns: ['customer_id', 'Product Name', 'Price', 'Quantity', 'Customer_Type', 'Order_Date', 'customer_email']


In [17]:
# Method 2: Rename specific columns
 # rename product_name
df_clean=df_clean.rename( columns={
    'Product Name':'product_name',
    'Order_Date':'order_date',
    'Price':'price',
    'Quantity':'quantity',
    'Customer_Type':'customer_type',
    'Customer Email':'customer_email',
})

Converting Data Type

In [18]:
# Check current data types 
print("Current data types:") 
print(df_clean.dtypes) 


Current data types:
customer_id                int64
product_name              object
price                    float64
quantity                   int64
customer_type             object
order_date        datetime64[ns]
customer_email            object
dtype: object


In [19]:
# Convert Price to float (already done above, but let's be explicit) 
df_clean['price'] = df_clean['price'].astype(float)

In [20]:
# Convert order_date to datetime 
df_clean['order_date'] = pd.to_datetime(df_clean['order_date'], errors='coerce')

 Cleaning Text Data 


In [21]:
# Clean product names 
print("Original product names:") 
print(df_clean['product_name'].unique()) 


Original product names:
['laptop' 'Smartphone' 'MOUSE' 'headphones' 'Tablet' 'smartphone'
 'keyboard' 'TABLET']


In [22]:
# Remove extra spaces and standardize case 
df_clean['product_name'] = df_clean['product_name'].str.strip() # Remove leading/trailing spaces df_clean['product_name'] = df_clean['product_name'].str.title() # Title Case 


In [24]:
df_clean['product_name'] = df_clean['product_name'].str.title() # Title Case

In [23]:
# Clean customer type 
print("\nOriginal customer types:") 
print(df_clean['customer_type'].unique()) 



Original customer types:
['PREMIUM' 'Regular' 'Premium' 'regular']


In [25]:
print("After cleaning:") 
print(df_clean['customer_type'].unique()) 


After cleaning:
['PREMIUM' 'Regular' 'Premium' 'regular']


In [26]:
# 2. calculate discounts (let's say 10% for premium customers)
def get_discount_rate(customer_type):
    if customer_type == 'Premium':
        return 0.10  # 10% discount for premium customers
    else:
        return 0.05 # No discount for regular customers

Business Logic Columns

In [27]:
# categorize coustomer spending
def categorize_spending(amount):
    if amount > 1000:
        return 'High'
    elif amount > 500:
        return 'Medium'
    else:
        return 'Low spender'

In [28]:
df_transformed["spending_category"]=df_transformed["final_"]

NameError: name 'df_transformed' is not defined

In [29]:
# 2. Create a product category column based on product name
def get_product_category(product_name):
product_lower = product_name.lower()
if 'laptop' in product_lower:
   return 'Computers'
elif 'smartphone' in product_lower:
   return 'Mobile'
elif 'tablet' in product_lower:
  return 'Mobile'
elif 'headphones' in product_lower:
  return 'Audio'
else:
  return 'Accessories'

IndentationError: expected an indented block after function definition on line 2 (2940570219.py, line 3)

️ Mapping and Replacing Values